# Result Arrangement of News Sentiment analysis

In [1]:
import pandas as pd
from glob import glob
import os
import pickle
import sys
import re
import dask.dataframe as dd
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report, roc_curve
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def CounterOutcome(row):
    from collections import Counter
    id_ = row.name
    target = row[['LogisticRegression',
       'NeuralNetwork_1', 'NeuralNetwork_2', 'RandomForestClassifier',
       'SVC', 'XGBoost']]
    counter = dict(Counter(target))
    counter['id'] = id_
    return counter

In [3]:
plt.rcParams["font.family"]

['NanumGothicOTF']

In [4]:
import Basic_Module as bm

## Path

In [5]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/nmodel/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
classifierPath = './data/pre_data/classifier/'
outpath = './outcome/'

## Load Data

In [21]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
naverData.set_index(keys='id', inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)
daumData.set_index(keys = 'id', inplace = True)
print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 10)
Daum : (9372, 10)


In [22]:
naverData.head()

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
id,,,,,,,,,,
5a29c445588c132954d1973a,정치,2017.12.07,연합뉴스,1713,1474,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{핵전쟁, 도화선, 고위, 미국, 중앙, 북한, 조선반도, 대변인}"
5a29c445588c132954d1973b,정치,2017.12.07,한국일보,2551,2121,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{증액, 지역구, 호남, 예산안, 의원, 정부안, 국민의당}"
5a29c445588c132954d1973c,정치,2017.12.07,뉴시스,610,540,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{정기국회, 구속영장 청구, 조사, 의원, 국정원장, 혐의, 검찰}"
5a29c445588c132954d1973d,정치,2017.12.07,연합뉴스,145,134,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{지명, 공직 사회, 법관, 후보자, 생활, 감사원장}"
5a29c445588c132954d1973e,정치,2017.12.07,동아일보,1074,937,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{시찰, 김정은, 사무차장, 접경, 양강도, 공장, 펠트먼, 훈련, 삼지연}"


In [23]:
daumData.head()

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
id,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{부작용, 연기, 세보 효과, 지역, 공장, 남성들, 건강}"
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{입냄새, 커피, 자세, 디스크, 치아, 식후, 건강, 점심 식사, 낮잠, 철분}"
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{경찰, 지인, 앙심, 남편, 범행, 주변, 철원, 아들, 성관계, 진술}"
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{네이처리퍼블릭, 신영자 이사장, 징역, 혐의, 매장, 유통업체, 검찰, 롯데}"
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{경찰, 범행, 과자, 혐의, 남자친구, 유치장, 폭행, 쪽지}"


In [6]:
outcomeDaumFileList = glob(outpath+'outcome_news_sentiment_analysis_daum'+'*')
outcomeNaverFileList = glob(outpath+'outcome_news_sentiment_analysis_naver'+'*')
print (len(outcomeDaumFileList), len(outcomeNaverFileList))

18 18


## Negative or Positive

In [7]:
outNaverDict = dict()
for idx, outcomeFile in enumerate(sorted(outcomeNaverFileList)):
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = 0, encoding='utf-8')
    out = df.apply(CounterOutcome, axis = 1)
    out = pd.DataFrame.from_records(out)
    out.rename({0 : 'negative', 1 : 'positive'}, axis = 1, inplace=True)
    out.set_index(keys='id', inplace = True)
    out.fillna(0, inplace=True)
    modelIs, tagger = bm.ExtractModelType(outcomeFile)
    mx = '{}_{}'.format(modelIs, tagger)
    print (mx)
    outNaverDict[mx] = out

Doc2Vec_dbow_twitter
Doc2Vec_dbow_mecab
Doc2Vec_dm-c_twitter
Doc2Vec_dm-c_mecab
Doc2Vec_dm-m_twitter
Doc2Vec_dm-m_mecab
fastText_CBOW_cbow_mean-0_twitter
fastText_CBOW_cbow_mean-0_mecab
fastText_CBOW_cbow_mean-1_twitter
fastText_CBOW_cbow_mean-1_mecab
fastText_skip-gram_cbow_mean-0_twitter
fastText_skip-gram_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-0_twitter
word2vec_CBOW_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-1_twitter
word2vec_CBOW_cbow_mean-1_mecab
word2vec_skip-gram_cbow_mean-0_twitter
word2vec_skip-gram_cbow_mean-0_mecab


In [8]:
outNaverDf = pd.DataFrame()
for idx,key in enumerate(outNaverDict):
    data = outNaverDict[key]
    print (key)
    if outNaverDf.shape == (0, 0):
        outNaverDf = data.copy()
    else:
        outNaverDf = outNaverDf.add(data)

Doc2Vec_dbow_twitter
Doc2Vec_dbow_mecab
Doc2Vec_dm-c_twitter
Doc2Vec_dm-c_mecab
Doc2Vec_dm-m_twitter
Doc2Vec_dm-m_mecab
fastText_CBOW_cbow_mean-0_twitter
fastText_CBOW_cbow_mean-0_mecab
fastText_CBOW_cbow_mean-1_twitter
fastText_CBOW_cbow_mean-1_mecab
fastText_skip-gram_cbow_mean-0_twitter
fastText_skip-gram_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-0_twitter
word2vec_CBOW_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-1_twitter
word2vec_CBOW_cbow_mean-1_mecab
word2vec_skip-gram_cbow_mean-0_twitter
word2vec_skip-gram_cbow_mean-0_mecab


In [9]:
outNaverDf.shape

(15120, 2)

In [24]:
mergedNaver = naverData.merge(outNaverDf, left_index=True, right_index= True)

In [30]:
mergedNaver['Decision'] = outNaverDf.idxmax(axis = 1)

In [31]:
mergedNaver.head()

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision
id,,,,,,,,,,,,,
5a29c445588c132954d1973a,정치,2017.12.07,연합뉴스,1713,1474,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{핵전쟁, 도화선, 고위, 미국, 중앙, 북한, 조선반도, 대변인}",86.0,22.0,negative
5a29c445588c132954d1973b,정치,2017.12.07,한국일보,2551,2121,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{증액, 지역구, 호남, 예산안, 의원, 정부안, 국민의당}",46.0,62.0,positive
5a29c445588c132954d1973c,정치,2017.12.07,뉴시스,610,540,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{정기국회, 구속영장 청구, 조사, 의원, 국정원장, 혐의, 검찰}",77.0,31.0,negative
5a29c445588c132954d1973d,정치,2017.12.07,연합뉴스,145,134,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{지명, 공직 사회, 법관, 후보자, 생활, 감사원장}",39.0,69.0,positive
5a29c445588c132954d1973e,정치,2017.12.07,동아일보,1074,937,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{시찰, 김정은, 사무차장, 접경, 양강도, 공장, 펠트먼, 훈련, 삼지연}",72.0,36.0,negative


In [33]:
mergedNaver.Decision.value_counts()

negative    8057
positive    7063
Name: Decision, dtype: int64

In [37]:
mergedNaver.to_csv('./outcome_for_News_sentiment_analysis/naver_news_sentiment_analysis.csv', index=True, header=True, encoding='utf-8')

In [11]:
outDaumDict = dict()
for idx, outcomeFile in enumerate(sorted(outcomeDaumFileList)):
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = 0, encoding='utf-8')
    out = df.apply(CounterOutcome, axis = 1)
    out = pd.DataFrame.from_records(out)
    out.rename({0 : 'negative', 1 : 'positive'}, axis = 1, inplace=True)
    out.set_index(keys='id', inplace = True)
    out.fillna(0, inplace=True)
    modelIs, tagger = bm.ExtractModelType(outcomeFile)
    mx = '{}_{}'.format(modelIs, tagger)
    print (mx)
    outDaumDict[mx] = out

Doc2Vec_dbow_twitter
Doc2Vec_dbow_mecab
Doc2Vec_dm-c_twitter
Doc2Vec_dm-c_mecab
Doc2Vec_dm-m_twitter
Doc2Vec_dm-m_mecab
fastText_CBOW_cbow_mean-0_twitter
fastText_CBOW_cbow_mean-0_mecab
fastText_CBOW_cbow_mean-1_twitter
fastText_CBOW_cbow_mean-1_mecab
fastText_skip-gram_cbow_mean-0_twitter
fastText_skip-gram_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-0_twitter
word2vec_CBOW_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-1_twitter
word2vec_CBOW_cbow_mean-1_mecab
word2vec_skip-gram_cbow_mean-0_twitter
word2vec_skip-gram_cbow_mean-0_mecab


In [12]:
outDaumDf = pd.DataFrame()
for idx,key in enumerate(outDaumDict):
    data = outDaumDict[key]
    print (key)
    if outDaumDf.shape == (0, 0):
        outDaumDf = data.copy()
    else:
        outDaumDf = outDaumDf.add(data)

Doc2Vec_dbow_twitter
Doc2Vec_dbow_mecab
Doc2Vec_dm-c_twitter
Doc2Vec_dm-c_mecab
Doc2Vec_dm-m_twitter
Doc2Vec_dm-m_mecab
fastText_CBOW_cbow_mean-0_twitter
fastText_CBOW_cbow_mean-0_mecab
fastText_CBOW_cbow_mean-1_twitter
fastText_CBOW_cbow_mean-1_mecab
fastText_skip-gram_cbow_mean-0_twitter
fastText_skip-gram_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-0_twitter
word2vec_CBOW_cbow_mean-0_mecab
word2vec_CBOW_cbow_mean-1_twitter
word2vec_CBOW_cbow_mean-1_mecab
word2vec_skip-gram_cbow_mean-0_twitter
word2vec_skip-gram_cbow_mean-0_mecab


In [13]:
outDaumDf.shape

(9372, 2)

In [26]:
mergedDaum = daumData.merge(outDaumDf, left_index=True, right_index= True)

In [28]:
mergedDaum['Decision'] = outDaumDf.idxmax(axis = 1)

In [29]:
mergedDaum.head()

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision
id,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{부작용, 연기, 세보 효과, 지역, 공장, 남성들, 건강}",50.0,58.0,positive
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{입냄새, 커피, 자세, 디스크, 치아, 식후, 건강, 점심 식사, 낮잠, 철분}",81.0,27.0,negative
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{경찰, 지인, 앙심, 남편, 범행, 주변, 철원, 아들, 성관계, 진술}",99.0,9.0,negative
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{네이처리퍼블릭, 신영자 이사장, 징역, 혐의, 매장, 유통업체, 검찰, 롯데}",77.0,31.0,negative
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{경찰, 범행, 과자, 혐의, 남자친구, 유치장, 폭행, 쪽지}",102.0,6.0,negative


In [34]:
mergedDaum.Decision.value_counts()

positive    4707
negative    4665
Name: Decision, dtype: int64

In [38]:
mergedDaum.to_csv('./outcome_for_News_sentiment_analysis/daum_news_sentiment_analysis.csv', index=True, header=True, encoding='utf-8')